In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../Data/Interim/hydrated_Tweet200316.csv', index_col=0)

In [9]:
df.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
377045    False
377046    False
377047    False
377048    False
377049    False
Length: 487972, dtype: bool

In [12]:
df.tweet

0         Large-scale testing for Covid-19 and wide avai...
1                                   Stories on covid_19 😱😱😱
2         Smart. More of this, please. https://t.co/c7Tk...
3         Not going to wear a bra for a month. 🙅🏻‍♀️✨\n\...
4         A COVID-19 coronavirus update from concerned p...
                                ...                        
377045    @charliekirk11 Democrats are using Covid19 to ...
377046    Can’t wait to be an old woman telling my grand...
377047    Y’all hate us sag so bad because we’re baddies...
377048    I’m scared we’re going to be unwilling or unab...
377049    @onhellmusic You didn't send me that mass text...
Name: tweet, Length: 487972, dtype: object

In [19]:
from sklearn.datasets import make_multilabel_classification
from sklearn.decomposition import LatentDirichletAllocation

In [20]:
X, _ = make_multilabel_classification(random_state=0)

In [21]:
lda = LatentDirichletAllocation(n_components=5, random_state=0)
lda.fit(X)

LatentDirichletAllocation(n_components=5, random_state=0)

In [22]:
lda.transform(X[:3])

array([[0.83264691, 0.15610504, 0.00372766, 0.00372946, 0.00379093],
       [0.11204968, 0.00563061, 0.00567138, 0.57851629, 0.29813204],
       [0.00361638, 0.15391126, 0.31659894, 0.52227005, 0.00360337]])

In [5]:
import nltk
import re

In [32]:
re.sub('\\u20aa', '', 'asdnjsadnk\u20aa')

'asdnjsadnk'

In [54]:
# Clean tweets
df['CleanTweet'] = [re.sub("[^a-zA-Z0-9\s,.-_´&%'\":€$£!?']", '',  re.sub(' http\S+', '', re.sub('\s',' ', tw))).replace(u'\xa0', u' ').lower() if isinstance(tw, str) else '' for tw in df.tweet]

# Get bigrams
bigrams_tweets = [list(nltk.bigrams(tw)) for tw in df['CleanTweet']]

In [110]:
unique_chars = sorted(''.join(set(''.join(df['CleanTweet'])))) # Assume all characters are in tweets from df1

unique_bigrams = [x+y for x in unique_chars for y in unique_chars]

bigram_mapper = dict(zip(unique_bigrams, range(len(unique_bigrams))))

In [145]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [116]:
vectorizer = CountVectorizer(vocabulary = bigram_mapper, ngram_range = (2,2), analyzer = "char")

In [117]:
X = vectorizer.fit_transform(df['CleanTweet'])

In [118]:
X.shape

(487972, 3481)

In [121]:
lda = LatentDirichletAllocation(n_components=5, verbose = 2, n_jobs = 3)
lda.fit(X[:10000])

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    6.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
iteration: 1 of max_iter: 10
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    4.5s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
iteration: 2 of max_iter: 10
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    4.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
iteration: 3 of max_iter: 10
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    4.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
iteration: 4 of max_iter: 10
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    4.4s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
iteration: 5 of max_iter: 10
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   

LatentDirichletAllocation(n_components=5, n_jobs=3, verbose=2)

In [136]:
topics = lda.transform(X[:10000]).argmax(axis = 1)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    4.2s finished


In [141]:
y = np.bincount(topics)
ii = np.nonzero(y)[0]
dict(zip(ii,y[ii]))

{0: 424, 1: 891, 2: 4264, 3: 2179, 4: 2242}

In [142]:
lda.transform(X[:10000])

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    3.5s finished


array([[0.16133972, 0.13859811, 0.69638492, 0.00183662, 0.00184063],
       [0.58946017, 0.01020377, 0.01020963, 0.01012878, 0.37999766],
       [0.00727604, 0.0072855 , 0.97082498, 0.00733381, 0.00727967],
       ...,
       [0.04489188, 0.00181722, 0.34178056, 0.39818723, 0.21332312],
       [0.12547465, 0.37938139, 0.12883567, 0.00448361, 0.36182468],
       [0.00400016, 0.11669848, 0.00396122, 0.18567844, 0.6896617 ]])

In [144]:
lda.score(X[:10000])

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    3.4s finished


-4636467.336107308

In [146]:
vectorizerTfidf = TfidfVectorizer(vocabulary = bigram_mapper, ngram_range = (2,2), analyzer = "char")

In [147]:
X = vectorizerTfidf.fit_transform(df['CleanTweet'])

In [148]:
lda = LatentDirichletAllocation(n_components=5, verbose = 2, n_jobs = 3)
lda.fit(X[:10000])

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    4.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
iteration: 1 of max_iter: 10
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
iteration: 2 of max_iter: 10
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
iteration: 3 of max_iter: 10
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
iteration: 4 of max_iter: 10
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
iteration: 5 of max_iter: 10
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   

LatentDirichletAllocation(n_components=5, n_jobs=3, verbose=2)

In [149]:
lda.score(X[:10000])

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished


-455997.85389304534

<5x3481 sparse matrix of type '<class 'numpy.float64'>'
	with 233 stored elements in Compressed Sparse Row format>